In [1]:
import pandas as pd
import numpy as np
import pymysql

In [2]:
from rake_nltk import Rake

In [3]:
from random import randint

In [4]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='',
                             db='times_shiksha2')

df = pd.read_sql('SELECT * FROM mdl_course', con=connection)

In [6]:
df.columns

Index(['id', 'category', 'sortorder', 'fullname', 'shortname', 'idnumber',
       'summary', 'summaryformat', 'format', 'showgrades', 'newsitems',
       'startdate', 'enddate', 'marker', 'maxbytes', 'legacyfiles',
       'showreports', 'visible', 'visibleold', 'groupmode', 'groupmodeforce',
       'defaultgroupingid', 'lang', 'calendartype', 'theme', 'timecreated',
       'timemodified', 'requested', 'enablecompletion', 'completionnotify',
       'cacherev'],
      dtype='object')

In [7]:
df = df.copy()[['id', 'fullname', 'summary']]

In [8]:
summarylist = []
for line in df['summary']:
    line = line.replace('\n','').replace('\r','').replace('\t','').lower()
    summarylist.append(line)
    
df['summary'] = summarylist

In [9]:
df['fullname'] = df['fullname'].str.lower()

In [10]:
df.head()

,id,fullname,summary
0,1,timesshiksha,a great place to start
1,6,microsoft word,microsoft word is agraphicalwordprocessing pro...
2,7,microsoft excel,microsoftexcelis aspreadsheet programusedto st...
3,8,microsoft powerpoint,power pointis apresentation program softwarepa...
4,12,powerful speaking,powerful speakingis an invaluable set of skill...


In [11]:
def concatenate_list(list):
    result= ''
    for element in list:
        result += ' ' + str(element)
    return result

In [12]:
# initializing the new column
df['Key_words'] = ""

for index, row in df.iterrows():
#     print(row)
#     print(index)
    plot = str(row['summary'])
#     print(plot)
    
    # instantiating Rake, by default it uses english stopwords from NLTK
    # and discards all puntuation characters as well
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)

    # getting the dictionary whith key words as keys and their scores as values
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column for the corresponding movie
    row['Key_words'] = concatenate_list(list(key_words_dict_scores.keys()))
#     print(row['Key_words'])
    df['Key_words'][index] = row['Key_words'].strip()
    
# dropping the Plot column
df.drop(columns = ['summary'], inplace = True)

C:\Users\mumtl5\AppData\Local\Continuum\anaconda3\envs\recommender\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [13]:
df['Key_words'][0]

'start great place'

In [14]:
# df.drop(columns = ['Timing'], inplace = True)

In [15]:
df.head()

,id,fullname,Key_words
0,1,timesshiksha,start great place
1,6,microsoft word,agraphicalwordprocessing program microsoft wor...
2,7,microsoft excel,microsoftexcelis aspreadsheet programusedto st...
3,8,microsoft powerpoint,inserted andformatted allows text display meth...
4,12,powerful speaking,deliver should1focus main message2keep tactful...


In [16]:
# df['desc'] = df[['Dept/Domain', 'Key_words']].apply(lambda x: ''.join(x.map(str)).lower(), axis=1)

In [17]:
df.head()

,id,fullname,Key_words
0,1,timesshiksha,start great place
1,6,microsoft word,agraphicalwordprocessing program microsoft wor...
2,7,microsoft excel,microsoftexcelis aspreadsheet programusedto st...
3,8,microsoft powerpoint,inserted andformatted allows text display meth...
4,12,powerful speaking,deliver should1focus main message2keep tactful...


In [18]:
# df['desc'][1]

In [19]:
# df['Program Name'] = df['Program Name'].apply(lambda x: str(x).lower())

In [20]:
# df3 = df.copy().drop(['Dept/Domain', 'Key_words'], axis=1)

In [21]:
# df2 = df.copy()

In [22]:
df.set_index(['fullname'], inplace=True)

In [23]:
df.head()

,id,Key_words
fullname,,
timesshiksha,1,start great place
microsoft word,6,agraphicalwordprocessing program microsoft wor...
microsoft excel,7,microsoftexcelis aspreadsheet programusedto st...
microsoft powerpoint,8,inserted andformatted allows text display meth...
powerful speaking,12,deliver should1focus main message2keep tactful...


In [24]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df['Key_words'])

# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [25]:
# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use in the function to match the indexes
indices = pd.Series(df.index)
# print(indices)
descs = pd.Series(df['Key_words'])

#  defining the function that takes in movie title 
# as input and returns the top 10 recommended movies
def get_recommendation(title, cosine_sim = cosine_sim):
    
    # initializing the empty list of recommendations
    recommendations = []
    target = False
    
    # gettin the index
    for i in indices:
        if title in i:
            target = i
    
    if target:
        idx = indices[indices == target].index[0]
        recommendations.append(dict({'id': df['id'][target], 'courseName': target.capitalize()}))
    else:
        idx = randint(0, len(indices))
        
    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    
    for i in top_10_indexes:
        recommendations.append(dict({'id': df['id'][list(df.index)[i]], 'courseName': list(df.index)[i].capitalize()}))
        
    return recommendations

In [26]:
def main():
    query = input()
    print(get_recommendation(query))

In [27]:
main()

excel
[{'id': 42, 'courseName': 'Excel 2016 pivot tables in depth'}, {'id': 7, 'courseName': 'Microsoft excel'}, {'id': 8, 'courseName': 'Microsoft powerpoint'}, {'id': 30, 'courseName': 'Marketing with facebook and twitter'}, {'id': 26, 'courseName': 'Microsoft dynamics crm customization config'}, {'id': 41, 'courseName': 'Complete beginners introduction to sql'}, {'id': 19, 'courseName': 'Adwords essential training'}, {'id': 32, 'courseName': 'Business-intellegence features in depth'}, {'id': 15, 'courseName': 'Cbt nuggets - mpls fundamentals'}, {'id': 16, 'courseName': 'Cbt nuggets microsoft sharepoint'}, {'id': 18, 'courseName': 'Creating a business plan'}]


In [28]:
df['id']['microsoft word']

6

In [29]:
df.index[6]

'cbt nuggets - mpls fundamentals'

In [33]:
str('http://timesshiksha.com/enrol/index.php?id='+ str(22))

'http://timesshiksha.com/enrol/index.php?id=22'